# 0. Environment setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Library to read json
import json

# Numeric and data manipulation tools
import pandas as pd
import numpy as np
import random

# Deep learning framework
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Natural language tools
import nltk
import gensim
import gensim.downloader as gloader

# Other tools
from tqdm.notebook import tqdm
from collections import OrderedDict
from time import time
from itertools import zip_longest

# Abstract classes
from abc import ABC, abstractmethod

# automatic mixed precision training:
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler

# Type hint
from typing import Optional, Callable, Tuple, Dict, List, Union

nltk.download('punkt')
nltk.download('stopwords')

# from sklearn.model_selection import train_test_split

# Use GPU acceleration if possible
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("using this device:", DEVICE)

if not(torch.cuda.is_available()):
  raise Exception("switch to runtime GPU, otherwise the code won't work properly")

[nltk_data] Downloading package punkt to /home/nihil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nihil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


using this device: cuda:0


In [3]:
# Set seed for reproducibility
def fix_random(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [4]:
# Use GPU acceleration if possible
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("using this device:", DEVICE)

# Define PAD token
PAD = '<PAD>'

using this device: cuda:0


In [5]:
# Lambda for computing the mean of a list
mean: Callable[[List[float]], float] = lambda l: sum(l) / len(l)

# Lambda for transforming a list of tuples into a tuple of lists
to_tuple_of_lists: Callable[[List[Tuple]], Tuple[List]] = lambda list_of_tuples: tuple(map(list, zip(*list_of_tuples)))

# Lambda for transforming a tuple of lists into a list of tuples
to_list_of_tuples: Callable[[Tuple[List]], List[Tuple]] = lambda tuple_of_lists: list(zip(*tuple_of_lists))

# Lambda for iterating with batches (if the length of the sequences does not match with the batch size, tuples of empty lists are appended)
batch_iteration: Callable[[List[Tuple]], zip] = lambda data, batch_size: zip_longest(*[iter(data)] * batch_size, fillvalue=([], [], ''))

# 1. Dataset preparation

In [6]:
"""
json structure:

data []
|---title
|---paragraphs []
|   |---context
|   |---qas []
|   |   |---answers []
|   |   |   |---answer_start
|   |   |   |---text
|   |   |---question
|   |   |---id
version

"""

filename = 'training_set.json'

with open(filename, 'r') as f:
    raw_data = f.readlines()[0]

parsed_data = json.loads(raw_data)['data']

context_list = []
context_index = -1
paragraph_index = -1

dataset = {'paragraph_index': [], 'context_index': [], 'question': [], 'answer_start': [], 'answer_end': [], 'answer_text': []}

for i in range(len(parsed_data)):
    paragraph_index += 1
    for j in range(len(parsed_data[i]['paragraphs'])):
        context_list.append(parsed_data[i]['paragraphs'][j]['context'])
        context_index += 1

        for k in range(len(parsed_data[i]['paragraphs'][j]['qas'])):
            question = parsed_data[i]['paragraphs'][j]['qas'][k]['question']

            for l in range(len(parsed_data[i]['paragraphs'][j]['qas'][k]['answers'])): 
                answer_start = parsed_data[i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start']
                answer_text = parsed_data[i]['paragraphs'][j]['qas'][k]['answers'][l]['text']

                answer_end = answer_start + len(answer_text)

                dataset['paragraph_index'].append(paragraph_index)
                dataset['context_index'].append(context_index)
                dataset['question'].append(question)
                dataset['answer_start'].append(answer_start)
                dataset['answer_end'].append(answer_end)
                dataset['answer_text'].append(answer_text)

df = pd.DataFrame.from_dict(dataset)

df.head()

,paragraph_index,context_index,question,answer_start,answer_end,answer_text
0,0,0,To whom did the Virgin Mary allegedly appear i...,515,541,Saint Bernadette Soubirous
1,0,0,What is in front of the Notre Dame Main Building?,188,213,a copper statue of Christ
2,0,0,The Basilica of the Sacred heart at Notre Dame...,279,296,the Main Building
3,0,0,What is the Grotto at Notre Dame?,381,420,a Marian place of prayer and reflection
4,0,0,What sits on top of the Main Building at Notre...,92,126,a golden statue of the Virgin Mary


In [7]:
# L'unica cosa che si potrebbe rimuovere è la fonetica, ma è poco presente. Inoltre facendolo bisognerebbe aggiornare gli indici

# Some examples of contexts and questions:
for i in range(0, 4000, 100):
    # print('Title:   ', title_list[df['title_index'][i]])
    print('Context: ', context_list[df['context_index'][i]])
    print('Question:', df['question'][i], "\n")

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? 

Context:  One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record

In [8]:
# Define split ratios
test_ratio = 0.2
val_ratio = 0.2

# Build array of paragraphs indexes and shuffle them
paragraph_indexes = df['paragraph_index'].unique()
np.random.shuffle(paragraph_indexes)
n_samples = len(paragraph_indexes)

# Reserve indexes for test set
test_size = int(test_ratio * n_samples)
train_val_size = n_samples - test_size
test_indexes = paragraph_indexes[-test_size:]
# Reserve indexes for validation set
val_size = int(val_ratio * train_val_size)
train_size = train_val_size - val_size
val_indexes = paragraph_indexes[-(test_size + val_size):-test_size]
# Reserve indexes for training set
train_indexes = paragraph_indexes[:train_size]

assert train_size == len(train_indexes), 'Something went wrong with train set slicing'
assert val_size == len(val_indexes), 'Something went wrong with val set slicing'
assert test_size == len(test_indexes), 'Something went wrong with test set slicing'

print('Number of train paragraphs:', train_size)
print('Number of validation paragraphs:', val_size)
print('Number of test paragraphs:', test_size)

# Split dataframe
df_train = df[np.in1d(df['paragraph_index'], train_indexes)]
df_val = df[np.in1d(df['paragraph_index'], val_indexes)]
df_test = df[np.in1d(df['paragraph_index'], test_indexes)]

print('\nNumber of train samples:', len(df_train))
print('Number of validation samples:', len(df_val))
print('Number of test samples:', len(df_test))

Number of train paragraphs: 284
Number of validation paragraphs: 70
Number of test paragraphs: 88

Number of train samples: 57451
Number of validation samples: 12921
Number of test samples: 17227


# 2. Word embeddings

In [9]:
print('Downloading GloVe model...')
emb_dim = 50
glove_model = gloader.load('glove-wiki-gigaword-' + str(emb_dim))
print('\nDownload completed.')


Download completed.


## Play with a little dataset

In [10]:
if True:
  df_train = df_train[:5]
  df_val = df_val[:5]
  df_test = df_test[:100]

In [11]:
def build_vocabulary(corpus: List[List[str]],
                     old_word_listing: Optional[List[str]] = None) -> (Dict[int, str], Dict[int, str], List[str]):
    
    flat_tokens = [x for sub in corpus for x in sub]
    
    if old_word_listing is None:  # standard case
        word_listing = [PAD] + list(OrderedDict.fromkeys(flat_tokens))
    else:  # case in which we extend an already existing vocabulary
        word_listing = list(OrderedDict.fromkeys(old_word_listing + flat_tokens))
        
    idx_to_word = {i: w for i, w in enumerate(word_listing)}
    word_to_idx = {w: i for i, w in enumerate(word_listing)}

    return idx_to_word, word_to_idx, word_listing

# Tokenize corpus
Contexts = df_train['context_index'].apply(lambda x: context_list[x])
X_trainC = Contexts.apply(lambda x: nltk.word_tokenize(x))
X_trainQ = df_train['question'].apply(lambda x: nltk.word_tokenize(x))
Y_train = [(start, end) for start, end in zip(df_train['answer_start'].tolist(), df_train['answer_end'].tolist())]

train_corpus = X_trainC.tolist() + X_trainQ.tolist()

Contexts = df_val['context_index'].apply(lambda x: context_list[x])
X_valC = Contexts.apply(lambda x: nltk.word_tokenize(x))
X_valQ = df_val['question'].apply(lambda x: nltk.word_tokenize(x))
Y_val = [(start, end) for start, end in zip(df_val['answer_start'].tolist(), df_val['answer_end'].tolist())]

val_corpus = X_valC.tolist() + X_valQ.tolist()

Contexts = df_val['context_index'].apply(lambda x: context_list[x])
X_testC = Contexts.apply(lambda x: nltk.word_tokenize(x))
X_testQ = df_test['question'].apply(lambda x: nltk.word_tokenize(x))
Y_test = [(start, end) for start, end in zip(df_test['answer_start'].tolist(), df_test['answer_end'].tolist())]

test_corpus = X_testC.tolist() + X_testQ.tolist()

# Get word mappings for each set
train_idx_to_word, train_word_to_idx, train_word_listing = build_vocabulary(train_corpus)
val_idx_to_word, val_word_to_idx, val_word_listing = build_vocabulary(val_corpus, old_word_listing=train_word_listing)
test_idx_to_word, test_word_to_idx, test_word_listing = build_vocabulary(test_corpus, old_word_listing=val_word_listing)

print('Words in training set:', len(train_word_listing))
print('Words in validation set:', len(val_word_listing))
print('Words in test set:', len(test_word_listing))

Words in training set: 100
Words in validation set: 183
Words in test set: 453


In [12]:
train_oov_words = [word for word in train_word_listing if word not in glove_model.vocab and word != PAD]
val_oov_words = [word for word in val_word_listing if word not in glove_model.vocab and word != PAD]
test_oov_words = [word for word in test_word_listing if word not in glove_model.vocab and word != PAD]

print(f'Total OOV terms in training set: {len(train_oov_words)} ({float(len(train_oov_words)) / len(train_word_listing) * 100:.2f}%)')
print(f'Total OOV terms in validation set: {len(val_oov_words)} ({float(len(val_oov_words)) / len(val_word_listing) * 100:.2f}%)')
print(f'Total OOV terms in test set: {len(test_oov_words)} ({float(len(test_oov_words)) / len(test_word_listing) * 100:.2f}%)')

Total OOV terms in training set: 33 (33.00%)
Total OOV terms in validation set: 42 (22.95%)
Total OOV terms in test set: 95 (20.97%)


In [13]:
def build_embedding_matrix(embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors,
                           word_to_idx: Dict[str, int],
                           oov_words: List[str],
                           old_embedding_matrix: Optional[np.ndarray] = None):
    # Initialize embedding matrix with all zeros
    embedding_matrix = np.zeros((len(word_to_idx), embedding_model.vector_size))
    
    # Helper function to analyze embeddings
    def analyze_embedding() -> (float, float):
        mean_list, std_list = [], []
        for word in tqdm(word_to_idx.keys(), leave=False):
            if word not in oov_words and word != PAD:
                embed = embedding_model[word]
                # Compute mean and std
                mean_list.append(np.mean(embed))
                std_list.append(np.std(embed))

        embedding_mean = mean(mean_list)
        embedding_std = mean(std_list)

        return embedding_mean, embedding_std
    
    # Get mean and std for the embeddings
    embedding_mean, embedding_std = analyze_embedding()

    for word, idx in tqdm(word_to_idx.items(), leave=False):
        # If word is PAD no action is performed (it will be assigned the zero vector)
        if word not in oov_words and word != PAD:
            embedding_matrix[idx] = embedding_model[word]
        elif word in oov_words:
            embedding_matrix[idx] = np.random.normal(loc=embedding_mean, scale=embedding_std, size=embedding_model.vector_size)
            
    return embedding_matrix

# Build embedding matrix based only on the training set (for training)
train_embedding_matrix = build_embedding_matrix(glove_model, train_word_to_idx, train_oov_words)
print('Shape of embedding matrix (training set):', train_embedding_matrix.shape)

# Build embedding matrix based on training + validation set (for validation)
val_embedding_matrix = build_embedding_matrix(glove_model, val_word_to_idx, val_oov_words, train_embedding_matrix)
print('Shape of embedding matrix (validation set):', val_embedding_matrix.shape)

# Build embedding matrix based on training + validation + test set (for test)
test_embedding_matrix = build_embedding_matrix(glove_model, test_word_to_idx, test_oov_words, val_embedding_matrix)
print('Shape of embedding matrix (test set):', test_embedding_matrix.shape)

Shape of embedding matrix (training set): (100, 50)
Shape of embedding matrix (validation set): (183, 50)
Shape of embedding matrix (test set): (453, 50)


In [14]:
from model.char_embedder import CharacterOneHotEncoder
from model.embedder import Embedder

'''class Embedder:
    def __init__(self,
                 word_embedding_matrix: np.ndarray,
                 char_embedding_matrix: np.ndarray,
                 word_to_idx: Dict[str, int]):
        self.word_embedding_matrix = word_embedding_matrix
        self.char_embedding_matrix = char_embedding_matrix
        self.word_to_idx = word_to_idx
    
    def get_word_embedding(self,
                           batch_word_seq: Tuple[List[str]],
                           max_len: Optional[int] = None) -> (torch.FloatTensor, torch.IntTensor, torch.IntTensor):
        if max_len is None:
          # Find maximum length of batch
          max_len = max(map(len, batch_word_seq))

        batch_word_embedding = []
        batch_char_embedding = []
        lengths = []
        for word_seq in batch_word_seq:
            if word_seq:  # Handle partial batches
                orig_len = len(word_seq)
                pad_len = max_len - orig_len
                # Keep track of original length
                lengths.append(orig_len)

                # Pad sequence
                padded_seq = word_seq + [PAD] * pad_len
                # Embed padded sequence
                word_embedded_seq, char_embedded_seq = list(zip(*[(self.word_embedding_matrix[self.word_to_idx[w]], self.char_embedding_matrix[self.word_to_idx[w]]) for w in padded_seq]))
                batch_word_embedding.append(word_embedded_seq)
                batch_char_embedding.append(char_embedded_seq)
        
        return torch.cuda.FloatTensor(batch_word_embedding, device=DEVICE), torch.cuda.IntTensor(batch_char_embedding, device=DEVICE), torch.cuda.IntTensor(lengths, device=DEVICE)
'''
train_onehot = CharacterOneHotEncoder(df_train, X_trainC.tolist())
val_onehot = CharacterOneHotEncoder(df_val, X_valC.tolist())
test_onehot = CharacterOneHotEncoder(df_test, X_testC.tolist())
# Create embedding helpers
train_embedder = Embedder(train_embedding_matrix, train_onehot, train_word_to_idx)
val_embedder = Embedder(val_embedding_matrix, val_onehot, val_word_to_idx)
test_embedder = Embedder(test_embedding_matrix, test_onehot, test_word_to_idx)

using this device: cuda:0


IndexError: list index out of range

## Prepare data for training

In [14]:
train_data = to_list_of_tuples((X_trainC.tolist(), X_trainQ.tolist(), Y_train))
val_data = to_list_of_tuples((X_valC.tolist(), X_valQ.tolist(), Y_val))
test_data = to_list_of_tuples((X_testC.tolist(), X_testQ.tolist(), Y_test))

# 3. Define model

# 4. Define training loop

In [18]:
def train(model: ClaimProver,
          data: List[Tuple[List[str], List[str], str]],
          batch_size: int,
          criterion: Callable[[torch.Tensor, torch.Tensor], float],
          optimizer: torch.optim,
          embedder: Embedder,
          max_len: Optional[int] = None,
          verbose: Optional[bool] = False,
          scaler: Optional[torch.cuda.amp.grad_scaler.GradScaler] = False) -> (float, float, float):
    loss_data = []
    correct = 0
    total = 0
    i = 0
    
    # Create batch iterator
    batch_iter = batch_iteration(data, batch_size)
    steps = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1
    if verbose:
        batch_iter = tqdm(batch_iter, total=steps, leave=False)
    for batch in batch_iter:
        
        # Extract samples
        (claims, evidences, labels) = to_tuple_of_lists(batch)
        
        # Get encoding of claims, evidences and labels, and move them to GPU if available
        claims_tensor, claims_lengths = embedder.get_word_embedding(claims, max_len)
        evidences_tensor, evidences_lengths = embedder.get_word_embedding(evidences, max_len)
        labels_tensor = torch.cuda.FloatTensor([l[0] for l in labels if len(l) > 0], device = DEVICE)   ################################ to predict only the start of the span
        
        if scaler:
           
            # Make prediction
            optimizer.zero_grad()
            with autocast():   #https://pytorch.org/docs/stable/notes/amp_examples.html
                scores = model(claims_tensor, evidences_tensor, claims_lengths, evidences_lengths)
                loss = criterion(scores, labels_tensor.unsqueeze(dim=1))
            # Backpropagation
        
            scaler.scale(loss).backward()  #https://pytorch.org/docs/stable/notes/amp_examples.html
            scaler.step(optimizer) #https://pytorch.org/docs/stable/notes/amp_examples.html
            scaler.update() #https://pytorch.org/docs/stable/notes/amp_examples.html
            
        else:
            # Make prediction
            optimizer.zero_grad()
            scores = model(claims_tensor, evidences_tensor, claims_lengths, evidences_lengths)
            loss = criterion(scores, labels_tensor.unsqueeze(dim=1))
            
            # Backpropagation
            loss.backward()  
            optimizer.step() 

        # Compute accuracy
        total_batch = scores.shape[0]
        predictions = torch.flatten(scores)
       
        if i == 0:
            print("Example of prediction vs label:",int(predictions[0].item()), int(labels_tensor[0].item()))
            i+=1
        correct_batch = (predictions.type(torch.int16)  == labels_tensor.type(torch.int16)).sum().item()
        
        # Update history
        loss_data.append(loss.item())
        correct += correct_batch
        total += total_batch
        

        
    return mean(loss_data), correct / total * 100

def evaluate(model: nn.Module,
             data: List[Tuple[List[str], List[str], str]],
             batch_size: int,
             criterion: Callable[[torch.Tensor, torch.Tensor], float],
             embedder: Embedder,
             label_to_idx: Dict[str, int],
             max_len: Optional[int] = None,
             verbose: Optional[bool] = False) -> (float, float, float):
    loss_data = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        # Create batch iterator
        batch_iter = batch_iteration(data, batch_size)
        steps = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1
        if verbose:
            batch_iter = tqdm(batch_iter, total=steps, leave=False)
        for batch in batch_iter:
            # Extract samples
            (claims, evidences, labels) = to_tuple_of_lists(batch)
            
            # Get encoding of claims, evidences and labels, and move them to GPU if available
            claims_tensor, claims_lengths = embedder.get_word_embedding(claims, max_len)
            evidences_tensor, evidences_lengths = embedder.get_word_embedding(evidences, max_len)
            labels_tensor = torch.cuda.FloatTensor([label_to_idx[l] for l in labels if len(l) > 0], device = DEVICE)
            
            # Make prediction
            scores = model(claims_tensor, evidences_tensor, claims_lengths, evidences_lengths)

            # Compute loss
            loss = criterion(scores, labels_tensor.unsqueeze(dim=1))
            
            # Compute accuracy
            total_batch = scores.shape[0]
            scores = torch.sigmoid(scores)
            predictions = scores.squeeze() > 0.5
            
            correct_batch = (int(predictions) == int(labels_tensor)).sum().item()
            
        
            # Update history
            loss_data.append(loss.item())
            correct += correct_batch

            total += total_batch
              
    return mean(loss_data), correct / total * 100
    

def training_loop(model: nn.Module,
                  train_data: List[Tuple[List[str], List[str]]],
                  optimizer: torch.optim,
                  epochs: int,
                  batch_size: int,
                  criterion: Callable[[torch.Tensor, torch.Tensor], float],
                  train_embedder: Embedder,
                  val_embedder: Optional[Embedder] = None,
                  max_len: Optional[int] = None,
                  lr_scheduler: torch.optim.lr_scheduler = None,
                  val_data: Optional[List[Tuple[List[str], List[str]]]] = None,
                  early_stopping: Optional[bool] = False,
                  patience: Optional[int] = 5,
                  tolerance: Optional[float] = 1e-4,
                  checkpoint_path: Optional[str] = None,
                  verbose: Optional[bool] = True,
                  seed: Optional[int] = 42,
                  mix_scale: Optional[bool] = True) -> (Dict[str, List[float]]):
    # Set seed for reproducibility
    if seed:
        random.seed(seed)

    history = {'loss': [],
               'accuracy': [],
               'val_loss': [],
               'val_accuracy': []}
    
    # Initialize variables for early stopping (if required)
    if val_data and val_embedder and early_stopping:
        min_val_loss = np.inf
        no_improve_counter = 0
    
    scaler = GradScaler() if mix_scale else False   # https://pytorch.org/docs/stable/notes/amp_examples.html
    
    for ep in range(epochs):
        if verbose:
            print('-' * 100)
            print(f'Epoch {ep + 1}/{epochs}')
        
        # Shuffle training set at each epoch
        random.shuffle(train_data)
        
        start = time()
        train_loss, train_accuracy = train(model, train_data, batch_size, criterion, optimizer, train_embedder, max_len, verbose, scaler)
        end = time()
        
        history['loss'].append(train_loss)
        history['accuracy'].append(train_accuracy)
        if verbose:
            print(f'\tLoss: {train_loss:.5f} - Accuracy: {train_accuracy:.2f}% [Time elapsed: {end - start:.2f} s]')
        
        # Do validation if required
        if val_data and val_embedder:
            
            # Shuffle validation set at each epoch
            random.shuffle(val_data)
            
            start = time()
            val_loss, val_accuracy = evaluate(model, val_data, batch_size, criterion, val_embedder, label_to_idx, max_len, verbose)
            end = time()
            
            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_accuracy)
            if verbose:
                print(f'\tValidation loss: {val_loss:.5f} - Validation accuracy: {val_accuracy:.2f}% [Time elapsed: {end - start:.2f} s]')
            
            if early_stopping and checkpoint_path:
                # If validation loss is lower than minimum, update minimum
                if val_loss < min_val_loss - tolerance:
                    min_val_loss = val_loss
                    no_improve_counter = 0
                    
                    # Save model
                    torch.save(model.state_dict(), checkpoint_path)
                # otherwise increment counter
                else:
                    no_improve_counter += 1
                # If loss did not improve for 'patience' epochs, break
                if no_improve_counter == patience:
                    if verbose:
                        print(f'Early stopping: no improvement in validation loss for {patience} epochs from {min_val_loss:.5f}')
                    # Restore model to best
                    model.load_state_dict(torch.load(checkpoint_path))
                    model.eval()
                    break
        
        # If lr scheduling is used, invoke next step
        if lr_scheduler:
            lr_scheduler.step()
    
    return history

# 5. Run training

In [22]:
# Epochs and batch size
EP = 1000
BS = 5
merge_strategy = 'concat'

# regression loss:
criterion = nn.MSELoss() 

# Build model
encoder = RNNAverage(input_size=emb_dim , hidden_size=32, bidirectional = True, num_layers =1).to(DEVICE)  # 32 * 2 = 64 due to bidirectionality
classifier_in_features = encoder.get_output_dim() * 2 if merge_strategy == 'concat' else encoder.get_output_dim()
classifier = nn.Linear(in_features=classifier_in_features, out_features=1)
prover = ClaimProver(encoder, classifier).to(DEVICE)

In [23]:
# Define Adam optimizer
optimizer = torch.optim.Adam(prover.parameters(), lr=1e-1)

history = training_loop(model=prover,
                        train_data=train_data,
                        optimizer=optimizer,
                        epochs=EP,
                        batch_size=BS,
                        criterion=criterion,
                        train_embedder=train_embedder,
                        val_embedder=val_embedder,
                        val_data=None,
                        early_stopping=True,
                        checkpoint_path='rnn_last.pt')


----------------------------------------------------------------------------------------------------
Epoch 1/1000
Example of prediction vs label: 0 92
	Loss: 106448.21094 - Accuracy: 0.00% [Time elapsed: 0.04 s]
----------------------------------------------------------------------------------------------------
Epoch 2/1000
Example of prediction vs label: 0 515
	Loss: 106448.21094 - Accuracy: 0.00% [Time elapsed: 0.04 s]
----------------------------------------------------------------------------------------------------
Epoch 3/1000
Example of prediction vs label: 0 279
	Loss: 106448.21094 - Accuracy: 0.00% [Time elapsed: 0.05 s]
----------------------------------------------------------------------------------------------------
Epoch 4/1000
Example of prediction vs label: 0 188
	Loss: 106448.21094 - Accuracy: 0.00% [Time elapsed: 0.05 s]
----------------------------------------------------------------------------------------------------
Epoch 5/1000
Example of prediction vs label: 0 2